In [45]:
import datetime as dt

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)
pd.options.display.float_format = "{:.4f}".format

import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import _fetch_batch, fetch_data, fetch_spot_data

In [46]:
# bnf = pd.read_csv('WAVE_1x_bnf.csv')
# nifty = pd.read_csv('WAVE_1x_nifty.csv')
# # fnf = pd.read_csv('MACD_fin_last.csv')
# midcp = pd.read_csv('WAVE_1x_midcp.csv')
# # bankex = pd.read_csv('MACD_bankex.csv')
# sensex = pd.read_csv('WAVE_1x_sensex.csv')

bnf = pd.read_csv('bnf-daily-wave.csv')
nifty = pd.read_csv('nifty-daily-wave.csv')
midcp = pd.read_csv('midcp-daily-wave.csv')
sensex = pd.read_csv('sensex-daily-wave.csv')

bnf['Index'] = 'BANKNIFTY'
nifty['Index'] = 'NIFTY'
# fnf['Index'] = 'FINNIFTY'
midcp['Index'] = 'MIDCPNIFTY'
# bankex['Index'] = 'BANKEX'
sensex['Index'] = 'SENSEX'

In [47]:
# combined_df = pd.concat([bnf, nifty, fnf, midcp, bankex, sensex], ignore_index=True)
combined_df = pd.concat([bnf, nifty, midcp, sensex], ignore_index=True)

In [48]:
combined_df.tail(30)

,Trade No.,Portfolio,Date,Entry Price,Exit Price,Points,Qty,Slippage,PnL,ROI%,Index
4858,326,1000000,2024-09-11,81939.8200,81586.3600,-353.4600,10,8.1880,-3616.4795,-0.3616,SENSEX
4859,327,1000000,2024-09-12,82440.2100,82749.6800,309.4700,10,1.2859,3081.8414,0.3082,SENSEX
4860,327,1000000,2024-09-13,82749.6800,82826.8800,77.2000,10,1.2859,759.1414,0.0759,SENSEX
4861,327,1000000,2024-09-16,82826.8800,82967.7100,140.8300,10,1.2859,1395.4414,0.1395,SENSEX
4862,327,1000000,2024-09-17,82967.7100,83043.1300,75.4200,10,1.2859,741.3414,0.0741,SENSEX
4863,327,1000000,2024-09-18,83043.1300,82870.1300,-173.0000,10,1.2859,-1742.8586,-0.1743,SENSEX
4864,327,1000000,2024-09-19,82870.1300,83257.0800,386.9500,10,1.2859,3856.6414,0.3857,SENSEX
4865,327,1000000,2024-09-20,83257.0800,84565.0400,1307.9600,10,1.2859,13066.7414,1.3067,SENSEX
4866,327,1000000,2024-09-23,84565.0400,84861.5500,296.5100,10,1.2859,2952.2414,0.2952,SENSEX
4867,327,1000000,2024-09-24,84861.5500,84937.4600,75.9100,10,1.2859,746.2414,0.0746,SENSEX


In [49]:
combined_df.head()

,Trade No.,Portfolio,Date,Entry Price,Exit Price,Points,Qty,Slippage,PnL,ROI%,Index
0,0,1000000,2018-01-01,25562.6500,25271.5500,-291.1000,45,5.1087,-13329.3926,-1.3329,BANKNIFTY
1,1,1000000,2018-01-04,25423.9500,25478.4000,54.4500,45,1.7033,2373.6011,0.2374,BANKNIFTY
2,1,1000000,2018-01-05,25478.4000,25619.9000,141.5000,45,1.7033,6290.8510,0.6291,BANKNIFTY
3,1,1000000,2018-01-08,25619.9000,25670.0000,50.1000,45,1.7033,2177.8510,0.2178,BANKNIFTY
4,2,1000000,2018-01-09,25688.8500,25688.0000,-0.8500,45,2.5668,-153.7575,-0.0154,BANKNIFTY


In [50]:
# combined_df.drop(columns=['Unnamed: 0'], inplace=True)

In [51]:
combined_df['Date'] = pd.to_datetime(combined_df['Date'])
combined_df = combined_df.sort_values(by='Date')

In [52]:
combined_df['Date'] = pd.to_datetime(combined_df['Date'])
combined_df['Trade Year'] = combined_df['Date'].dt.year

In [53]:
# ONLY NSE
combined_df['Portfolio'] = 1000000
combined_df.loc[(combined_df['Trade Year'] <= 2017), 'Allocation'] = 500000
combined_df.loc[(combined_df['Trade Year'] > 2017) & (combined_df['Trade Year'] <= 2021), 'Allocation'] = 333333
combined_df.loc[(combined_df['Trade Year'] >= 2022), 'Allocation'] = 250000

# NSE + BSE
# combined_df['Portfolio'] = 60000000
# combined_df.loc[(combined_df['Trade Year'] <= 2017), 'Allocation'] = 12000000
# combined_df.loc[(combined_df['Trade Year'] > 2017) & (combined_df['Trade Year'] < 2022), 'Allocation'] = 12000000
# combined_df.loc[(combined_df['Trade Year'] >= 2022), 'Allocation'] = 10000000

In [54]:
combined_df['Qty apAlloc'] = combined_df['Qty'] * (combined_df['Allocation'] / 1000000)
# combined_df['Qty apAlloc'] = combined_df['Qty']

In [55]:
combined_df['PnL apAlloc'] = (combined_df['Points'] - combined_df['Slippage']) * combined_df['Qty apAlloc']

In [56]:
combined_df['ROI% PF'] = combined_df['PnL apAlloc'] * 100 / combined_df['Portfolio']

In [57]:
# combined_df['ROI% PF'] = combined_df['PnL'] * 100 / combined_df['Portfolio']

In [58]:
# combined_df['ROI% PF'] /= 5

In [59]:
def generate_stats(tb_expiry):
    stats_df8 = pd.DataFrame(
        index=range(2017, 2025),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            
            "Max Drawdown",
            "ROI/DD Ratio",
            
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2017, 2025):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI% PF"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI% PF"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI% PF"] > 0]["ROI% PF"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI% PF"] < 0]["ROI% PF"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI% PF"].cumsum() - year_trades["ROI% PF"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        # variation = f'{signal_ma} , {trailing_ma}, {time_of_day}'
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            # variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI% PF"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI% PF"] > 0]["ROI% PF"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI% PF"] < 0]["ROI% PF"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI% PF"].cumsum() - combined_df_sorted["ROI% PF"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    # overall_variation = variation
    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        # overall_variation,
    ]
    return pd.DataFrame(stats_df8)

In [60]:
stats = generate_stats(combined_df)
stats

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,4.4946,207,51.2077,0.1928,-0.1578,-1.8429,2.4388
2018,11.8299,573,57.4171,0.1711,-0.1822,-5.7281,2.0652
2019,8.1791,575,51.8261,0.1918,-0.1768,-8.7331,0.9366
2020,28.3863,601,55.7404,0.3403,-0.3219,-11.0201,2.5759
2021,18.8252,610,55.0820,0.2178,-0.1983,-8.7740,2.1456
2022,16.3398,754,54.1114,0.1841,-0.1698,-10.6216,1.5383
2023,14.6571,839,57.2110,0.1167,-0.1152,-4.7038,3.1160
2024,5.6003,729,54.4582,0.1234,-0.1307,-6.3218,0.8859
Overall,108.3123,4888.0000,55.0123,0.1864,-0.1786,-11.0201,9.8286


# Stats for WAVE 1x Lev-> BNF, NF, SENSEX, MIDCP

In [39]:
stats = generate_stats(combined_df)
stats

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,25.1956,104,34.6154,1.8572,-0.6127,-6.1667,4.0858
2018,56.1707,201,38.3085,1.7350,-0.6244,-21.4809,2.6149
2019,49.7535,231,32.9004,1.8628,-0.5924,-21.9333,2.2684
2020,185.9565,216,37.5000,3.9514,-0.9934,-16.2184,11.4658
2021,99.0073,226,38.4956,2.1413,-0.6279,-24.3461,4.0667
2022,64.7539,211,36.0190,2.1629,-0.7380,-18.7507,3.4534
2023,52.3752,232,35.3448,1.5879,-0.5189,-13.5865,3.8549
2024,23.5894,193,38.3420,1.4530,-0.7053,-17.6492,1.3366
Overall,556.8021,1614.0000,36.4932,2.1231,-0.6768,-24.3461,22.8703


# Only NSE Indices

In [16]:
stats = generate_stats(combined_df)
stats

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,79.6472,106,34.9057,4.0482,-1.0165,-15.4559,5.1532
2018,51.0977,209,38.2775,2.0236,-0.8588,-33.5205,1.5244
2019,77.4880,234,33.7607,2.4183,-0.7326,-29.7556,2.6041
2020,239.2211,221,38.4615,4.8860,-1.2948,-18.1667,13.1681
2021,119.3993,242,40.0826,2.5267,-0.8669,-30.8842,3.8660
2022,115.0063,286,37.7622,2.6876,-0.9845,-30.3764,3.7860
2023,110.7106,296,35.1351,2.1957,-0.6127,-24.9829,4.4315
2024,62.7774,245,39.5918,1.9800,-0.8735,-22.0141,2.8517
Overall,855.3476,1839.0000,37.3573,2.7275,-0.8841,-33.5205,25.5171


In [44]:
combined_df.to_csv('Wave 1x Daily PnL combined.csv', index=False)

In [61]:
combined_df.tail(30)

,Trade No.,Portfolio,Date,Entry Price,Exit Price,Points,Qty,Slippage,PnL,ROI%,Index,Trade Year,Allocation,Qty apAlloc,PnL apAlloc,ROI% PF
3067,386,1000000,2024-11-19,23750.0000,23468.7000,-281.3000,50,9.4686,-14538.4325,-1.4538,NIFTY,2024,250000.0000,12.5000,-3634.6081,-0.3635
3643,172,1000000,2024-11-21,12163.5500,12166.6500,3.1000,100,12.1234,-902.3410,-0.0902,MIDCPNIFTY,2024,250000.0000,25.0000,-225.5853,-0.0226
1607,893,1000000,2024-11-22,50964.1000,51098.5500,134.4500,15,3.4353,1965.2199,0.1965,BANKNIFTY,2024,250000.0000,3.7500,491.3050,0.0491
3068,387,1000000,2024-11-22,23657.0000,23873.5000,216.5000,50,1.9055,10729.7273,1.0730,NIFTY,2024,250000.0000,12.5000,2682.4318,0.2682
3644,173,1000000,2024-11-22,12248.4500,12309.7500,61.3000,100,2.4797,5882.0260,0.5882,MIDCPNIFTY,2024,250000.0000,25.0000,1470.5065,0.1471
4881,334,1000000,2024-11-22,78362.9300,78957.4300,594.5000,10,3.1510,5913.4903,0.5913,SENSEX,2024,250000.0000,2.5000,1478.3726,0.1478
3069,387,1000000,2024-11-25,23873.5000,24250.8000,377.3000,50,1.9055,18769.7273,1.8770,NIFTY,2024,250000.0000,12.5000,4692.4318,0.4692
3645,173,1000000,2024-11-25,12309.7500,12590.6500,280.9000,100,2.4797,27842.0260,2.7842,MIDCPNIFTY,2024,250000.0000,25.0000,6960.5065,0.6961
1608,893,1000000,2024-11-25,51098.5500,52199.4000,1100.8500,15,3.4353,16461.2199,1.6461,BANKNIFTY,2024,250000.0000,3.7500,4115.3050,0.4115
4882,334,1000000,2024-11-25,78957.4300,80171.5400,1214.1100,10,3.1510,12109.5903,1.2110,SENSEX,2024,250000.0000,2.5000,3027.3976,0.3027
